In [1]:
import wave_networks.test as test

test.hello_world()

Hello World


In [2]:
# https://soundata.readthedocs.io/en/latest/source/tutorial.html
import soundata
urbansound8k = soundata.initialize('urbansound8k', data_home='/homes/55/bwilop/gdl/ablation_study_wavelet_networks/data')
# urbansound8k.download()  
# urbansound8k.validate() 
clip = urbansound8k.choice_clip()  
print(clip)  
print(len(urbansound8k.clip_ids))

Clip(
  audio_path="/homes/55/bwilop/gdl/ablation_study_wavelet_networks/data/audio/fold5/139665-9-0-23.wav",
  clip_id="139665-9-0-23",
  audio: The clip's audio
            * np.ndarray - audio signal
            * float - sample rate,
  class_id: The clip's class id.
            * int - integer representation of the class label (0-9). See Dataset Info in the documentation for mapping,
  class_label: The clip's class label.
            * str - string class name: air_conditioner, car_horn, children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, street_music,
  fold: The clip's fold.
            * int - fold number (1-10) to which this clip is allocated. Use these folds for cross validation,
  freesound_end_time: The clip's end time in Freesound.
            * float - end time in seconds of the clip in the original freesound recording,
  freesound_id: The clip's Freesound ID.
            * str - ID of the freesound.org recording from which this clip was taken,

In [3]:
clip.audio

(array([0.06288499, 0.10498973, 0.13549979, ..., 0.0999035 , 0.09920264,
        0.08016797], dtype=float32),
 44100)

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pywt

def plot_waveform_and_wavelet(clip, wavelet='morl'):
    """
    Plots the waveform and the continuous wavelet transform (CWT) of an audio clip.
    
    The wavelet transform is computed using scales that are powers of 2 (i.e. 1, 2, 4, 8, ...)
    up to the maximal power of 2 that is still smaller than the length of the audio signal.
    
    Parameters
    ----------
    clip : soundata clip object
        A clip object returned by soundata.initialize(...).choice_clip() or similar.
        It must provide clip.audio which returns a tuple (signal, sample_rate).
    wavelet : str, optional
        The type of wavelet to use for the CWT (default is 'morl').
    """
    # Extract audio samples and sampling rate
    signal, sr = clip.audio
    time_axis = np.linspace(0, len(signal) / sr, num=len(signal))
    
    # Determine the maximum power of 2 that is smaller than the length of the signal.
    max_power = int(math.floor(math.log2(len(signal))))
    # Ensure that 2**max_power is strictly less than len(signal)
    if 2**max_power >= len(signal):
        max_power -= 1
    # Define scales: powers of 2 from 2**0 up to 2**max_power
    max_power = min(20, max_power)
    scales = [2**i for i in range(max_power + 1)]
    
    # Compute the CWT using the specified wavelet and the defined scales
    coeffs, freqs = pywt.cwt(signal, scales, wavelet, sampling_period=1/sr)
    log_coeffs = np.log10(np.abs(coeffs) + 0.1)
    
    # Create a figure with two subplots: one for the waveform and one for the wavelet transform
    fig, axs = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # Plot the waveform on the top subplot
    axs[0].plot(time_axis, signal, color='black')
    axs[0].set_title("Audio Waveform")
    axs[0].set_ylabel("Amplitude")
    axs[0].grid(True)
    
    # Plot the wavelet transform on the bottom subplot
    im = axs[1].imshow(
        log_coeffs,
        extent=[time_axis[0], time_axis[-1], scales[-1], scales[0]],
        cmap='viridis',
        aspect='auto'
    )
    axs[1].invert_yaxis()  # so that lower scales (higher frequencies) are on top
    axs[1].set_title(f"Continuous Wavelet Transform (Wavelet: {wavelet})")
    axs[1].set_xlabel("Time (s)")
    axs[1].set_ylabel("Scale (powers of 2)")
    fig.colorbar(im, ax=axs[1], label='Magnitude')
    
    plt.tight_layout()
    plt.show()


plot_waveform_and_wavelet(clip, wavelet='morl')